<a href="https://colab.research.google.com/github/jamslaugh/Data-Science-Projects/blob/main/distributed_hyperopt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%bash
pip install -U sentence-transformers
pip install -U hdbscan
pip install -U umap-learn

  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=80f38858cffb39c1d37d1f30b747d635d3164dd5098d7c3004e42bb101db974f
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for hdbscan: filename=hdbscan-0.8.28-cp37-cp37m-linux_x86_64.whl size=2330845 sha256=80154c0229dd85b88af294cf368ccfe43fd68e4a2e27ca08f116d0c521d7429d
  Stored in directory: /root/.cache/pip/whee

In [ ]:
!wget https://raw.githubusercontent.com/PolyAI-LDN/task-specific-datasets/master/banking_data/train.csv

--2022-04-08 21:01:16--  https://raw.githubusercontent.com/PolyAI-LDN/task-specific-datasets/master/banking_data/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 839073 (819K) [text/plain]
Saving to: ‘train.csv’

train.csv           100%[===================>] 819.41K  --.-KB/s    in 0.04s   

2022-04-08 21:01:16 (18.4 MB/s) - ‘train.csv’ saved [839073/839073]



In [ ]:
!pip install --upgrade hyperopt

     |████████████████████████████████| 1.6 MB 5.6 MB/s 
     |████████████████████████████████| 199 kB 32.8 MB/s 
  Attempting uninstall: hyperopt
    Found existing installation: hyperopt 0.1.2
    Uninstalling hyperopt-0.1.2:
      Successfully uninstalled hyperopt-0.1.2


In [ ]:
!ls -llah

total 836K
drwxr-xr-x 1 root root 4.0K Apr  8 21:01 .
drwxr-xr-x 1 root root 4.0K Apr  8 20:59 ..
drwxr-xr-x 4 root root 4.0K Mar 23 14:21 .config
drwxr-xr-x 1 root root 4.0K Mar 23 14:22 sample_data
-rw-r--r-- 1 root root 820K Apr  8 21:01 train.csv


In [ ]:
!apt-get install openjdk-8-jdk-headless > /dev/null

In [ ]:
!wget https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz

--2022-04-08 21:01:36--  https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 300971569 (287M) [application/x-gzip]
Saving to: ‘spark-3.2.1-bin-hadoop3.2.tgz’

spark-3.2.1-bin-had 100%[===================>] 287.03M   160MB/s    in 1.8s    

2022-04-08 21:01:38 (160 MB/s) - ‘spark-3.2.1-bin-hadoop3.2.tgz’ saved [300971569/300971569]



In [ ]:
!ls -llah

total 288M
drwxr-xr-x 1 root root 4.0K Apr  8 21:01 .
drwxr-xr-x 1 root root 4.0K Apr  8 20:59 ..
drwxr-xr-x 4 root root 4.0K Mar 23 14:21 .config
drwxr-xr-x 1 root root 4.0K Mar 23 14:22 sample_data
-rw-r--r-- 1 root root 288M Jan 20 21:37 spark-3.2.1-bin-hadoop3.2.tgz
-rw-r--r-- 1 root root 820K Apr  8 21:01 train.csv


In [ ]:
!pip install -q findspark

In [ ]:
!tar xf spark-3.2.1-bin-hadoop3.2.tgz

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"

In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [ ]:
!wget --continue https://raw.githubusercontent.com/GarvitArya/pyspark-demo/main/sample_books.json -O /tmp/sample_books.json

--2022-04-08 21:26:22--  https://raw.githubusercontent.com/GarvitArya/pyspark-demo/main/sample_books.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 416 Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



In [ ]:
df = spark.read.json("/tmp/sample_books.json")

In [ ]:
df.show(5)

+--------------------+--------------+-----+----------------+------------+
|              author|       edition|price|           title|year_written|
+--------------------+--------------+-----+----------------+------------+
|        Austen, Jane|       Penguin| 18.2|Northanger Abbey|        1814|
|        Tolstoy, Leo|       Penguin| 12.7|   War and Peace|        1865|
|        Tolstoy, Leo|       Penguin| 13.5|   Anna Karenina|        1875|
|     Woolf, Virginia|Harcourt Brace| 25.0|   Mrs. Dalloway|        1925|
|Cunnningham, Michael|Harcourt Brace|12.35|       The Hours|        1999|
+--------------------+--------------+-----+----------------+------------+
only showing top 5 rows



In [ ]:
from hyperopt import hp, Trials, partial, fmin, tpe, STATUS_OK
import hyperopt
from umap import UMAP
import hdbscan
import pynndescent
import numpy as np

In [ ]:
from pyspark.sql import functions as F
df = df.withColumn('author_last_name',F.split(df.author,", ")[0])

In [ ]:
df.show()

+--------------------+-----------------+-----+--------------------+------------+----------------+
|              author|          edition|price|               title|year_written|author_last_name|
+--------------------+-----------------+-----+--------------------+------------+----------------+
|        Austen, Jane|          Penguin| 18.2|    Northanger Abbey|        1814|          Austen|
|        Tolstoy, Leo|          Penguin| 12.7|       War and Peace|        1865|         Tolstoy|
|        Tolstoy, Leo|          Penguin| 13.5|       Anna Karenina|        1875|         Tolstoy|
|     Woolf, Virginia|   Harcourt Brace| 25.0|       Mrs. Dalloway|        1925|           Woolf|
|Cunnningham, Michael|   Harcourt Brace|12.35|           The Hours|        1999|     Cunnningham|
|         Twain, Mark|          Penguin| 5.76|    Huckleberry Finn|        1865|           Twain|
|    Dickens, Charles|     Random House| 5.75|         Bleak House|        1870|         Dickens|
|         Twain, Mar

In [ ]:
df.printSchema()

root
 |-- author: string (nullable = true)
 |-- edition: string (nullable = true)
 |-- price: double (nullable = true)
 |-- title: string (nullable = true)
 |-- year_written: long (nullable = true)
 |-- author_last_name: string (nullable = true)



In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from sentence_transformers import SentenceTransformer
import pandas as pd
# load intents dataset
data_sample = spark.read.option("delimiter", ",").option("header", "true").csv('./train.csv')
data_sample = data_sample.toPandas()
all_intents = list(data_sample['text'])

# define the document embedding models to use for comparison
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model_use = hub.load(module_url)
model_st1 = SentenceTransformer('all-mpnet-base-v2')
model_st2 = SentenceTransformer('all-MiniLM-L6-v2')
model_st3 = SentenceTransformer('paraphrase-mpnet-base-v2')

def embed(model, model_type, sentences):
    """
    wrapper function for generating message embeddings
    """
  
    if model_type == 'use':
        embeddings = model(sentences)
    elif model_type == 'sentence transformer':
        embeddings = model.encode(sentences)

    return embeddings

# generate embeddings for each model
embeddings_use = embed(model_use, 'use', all_intents)
embeddings_st1 = embed(model_st1, 'sentence transformer', all_intents)
embeddings_st2 = embed(model_st2, 'sentence transformer', all_intents)
embeddings_st3 = embed(model_st3, 'sentence transformer', all_intents)

In [ ]:
def generate_clusters(message_embeddings,
                      n_neighbors,
                      n_components, 
                      min_cluster_size,
                      random_state = None):
    """
    Generate HDBSCAN cluster object after reducing embedding dimensionality with UMAP
    """
    
    umap_embeddings = UMAP(n_neighbors=n_neighbors, 
                            n_components=n_components, 
                            metric='cosine', 
                            random_state=random_state).fit_transform(message_embeddings)

    clusters = hdbscan.HDBSCAN(min_cluster_size = min_cluster_size,
                               metric='euclidean', 
                               cluster_selection_method='eom').fit(umap_embeddings)

    return clusters

In [ ]:
def score_clusters(clusters, prob_threshold = 0.05):
    """
    Returns the label count and cost of a given cluster supplied from running hdbscan
    """
    
    cluster_labels = clusters.labels_
    label_count = len(np.unique(cluster_labels))
    total_num = len(clusters.labels_)
    cost = (np.count_nonzero(clusters.probabilities_ < prob_threshold)/total_num)
    
    return label_count, cost

In [ ]:
def objective(params, embeddings, label_lower, label_upper):
    """
    Objective function for hyperopt to minimize, which incorporates constraints
    on the number of clusters we want to identify
    """
    
    clusters = generate_clusters(embeddings, 
                                 n_neighbors = params['n_neighbors'], 
                                 n_components = params['n_components'], 
                                 min_cluster_size = params['min_cluster_size'],
                                 random_state = params['random_state'])
    
    label_count, cost = score_clusters(clusters, prob_threshold = 0.05)
    
    #15% penalty on the cost function if outside the desired range of groups
    if (label_count < label_lower) | (label_count > label_upper):
        penalty = 0.15 
    else:
        penalty = 0
    
    loss = cost + penalty
    
    return {'loss': loss, 'label_count': label_count, 'status': STATUS_OK}

In [ ]:
def bayesian_search(embeddings, space, label_lower, label_upper, max_evals=100):
    """
    Perform bayseian search on hyperopt hyperparameter space to minimize objective function
    """
    
    trials = Trials()
    fmin_objective = partial(objective, embeddings=embeddings, label_lower=label_lower, label_upper=label_upper)
    best = fmin(fmin_objective, 
                space = space, 
                algo=tpe.suggest,
                max_evals=max_evals, 
                trials=hyperopt.SparkTrials(3))

    best_params = space_eval(space, best)
    print ('best:')
    print (best_params)
    print (f"label count: {trials.best_trial['result']['label_count']}")
    
    best_clusters = generate_clusters(embeddings, 
                                      n_neighbors = best_params['n_neighbors'], 
                                      n_components = best_params['n_components'], 
                                      min_cluster_size = best_params['min_cluster_size'],
                                      random_state = best_params['random_state'])
    
    return best_params, best_clusters, trials

In [ ]:
#fundamental to make UMAP work with spark 
pynndescent.rp_trees.FlatTree.__module__  = "pynndescent.rp_trees"
hspace = {
    "n_neighbors": hp.choice('n_neighbors', range(3,16)),
    "n_components": hp.choice('n_components', range(3,16)),
    "min_cluster_size": hp.choice('min_cluster_size', range(2,16)),
    "min_samples": None,
    "random_state": 42
}

label_lower = 30
label_upper = 100
max_evals = 100

bayesian_search(embeddings_st1,
                space=hspace,
                label_lower=label_lower, 
                label_upper=label_upper, 
                max_evals=max_evals)

  7%|▋         | 7/100 [04:17<57:05, 36.83s/trial, best loss: 0.29829239065308566]


Total Trials: 10: 10 succeeded, 0 failed, 0 cancelled.
INFO:hyperopt-spark:Total Trials: 10: 10 succeeded, 0 failed, 0 cancelled.


KeyboardInterrupt: ignored

In [ ]:
hyperopt.__version__

In [ ]:
best_params = {
    "n_neighbors": 16,
    "n_components": 8,
    "min_cluster_size": 4,
    "min_samples": 3,
    "random_state": 42
}

In [ ]:
embeddings_st1

In [ ]:
best_clusters = generate_clusters(embeddings_st1, 
                                      n_neighbors = best_params['n_neighbors'], 
                                      n_components = best_params['n_components'], 
                                      min_cluster_size = best_params['min_cluster_size'],
                                      random_state = best_params['random_state'])